In [11]:
import sys
import os
import cv2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
# from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [3]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
num_classes = 29
nb_epochs = 10
img_sz = (224, 224)

# data paths
train_path = '../../data/asl_alphabet_train/'
validation_path = '../../data/asl_alphabet_validation/'

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical',
        subset='training')
test_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical',
        subset='validation')
validation_generator = valid_datagen.flow_from_directory(
        validation_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')

Found 60901 images belonging to 29 classes.
Found 26100 images belonging to 29 classes.
Found 30 images belonging to 29 classes.


In [5]:
next(train_generator)[0].shape

(32, 224, 224, 3)

In [13]:
new_in = keras.Input(shape=next(train_generator)[0].shape[1:])
mobile = MobileNetV2(weights='imagenet', include_top=False, input_tensor=new_in)

x = mobile.layers[-6].output
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(mobile.input, predictions)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [14]:
for layer in mobile.layers[:-23]:
    layer.trainable = False


model.compile(optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                           min_delta=0,
                                           patience=1,
                                           verbose=0,
                                           mode="auto",
                                           baseline=None,
                                           restore_best_weights=False)

callbacks = [early_stop]
t=time.time()
historytemp = model.fit(train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=5,
                        validation_data=validation_generator,
                        callbacks=callbacks)
print('Training time: %s' % (t - time.time()))

Epoch 1/5


ValueError: in user code:

    /Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(

In [ ]:
model.save('models/model.h5')